<a href="https://colab.research.google.com/github/Anthei0774/Advent-of-Code-2021/blob/main/day_8_seven_segment_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
####################################################################################################
### read & process

lines = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

f = open("8.txt")
lines = f.read()
f.close()

lines = lines.split("\n")
lines = [ l.split(" | ") for l in lines ]

####################################################################################################
### PART 1

### right now we don't need to decode any of the falsely printed outputs
### task: count those words, where the output is 1, 4, 7 or 8 (with lengths 2, 4, 3, 7)
ans = 0
for l in lines:

    o = l[1].split(" ")
    # print(o)

    for w in o:
        if len(w) in [2, 4, 3, 7]:
            ans += 1
            # print(w)

print("Answer:", ans)

####################################################################################################
### PART 2

### decoding time!!!

# init two dictionaries: mapping from string wires to digits and vice versa
wire_to_digit = []

# Init: the wires of 1, 4, 7 and 8 can be identified by length only - previous task
for i, l in enumerate(lines):

    wires = l[0].split(" ")
    wires = [ "".join(sorted(w)) for w in wires ]

    # sort wires first by alphabetical, then by length - more clear mapping
    # https://stackoverflow.com/questions/4659524/how-to-sort-by-length-of-string-followed-by-alphabetical-order - python sort is stable :)
    wires = sorted(wires)
    wires = sorted(wires, key = len)

    # get wires for 1, 4, 7 and 8
    wire_to_digit.append({ w : 1 if len(w) == 2 else 4 if len(w) == 4 else 7 if len(w) == 3 else 8 if len(w) == 7 else -1 for w in wires })

# for mapping in wire_to_digit:
#     print(mapping)
    
### Now get the remaining mystery... :o

for i in range(len(wire_to_digit)):

    # Observation I: the wires of 0, 6, 9 have length of 6. From these, the wires of 6 does not contain the wires of 1, therefore match the wires of 1 with the wires of 0, 6, 9. The lowest matching corresponds to 6.
    wire_1 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 2][0]
    wires_069 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 6]
    
    matching = [ len(w.intersection(wire_1)) for w in wires_069 ]
    idx = matching.index(1)

    wire_6 = wires_069[idx]
    wire_6 = "".join(sorted(list(wire_6)))

    wire_to_digit[i][wire_6] = 6

    # Observation II: apply the previous logic => the wires of 4 are entirely in the wires of 9, but not the same in 0 => 4 can be found.
    wire_4 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 4][0]
    wires_09 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 6 and wire_to_digit[i][wires] == -1] # do not forget to exclude 6 here
    
    matching = [ len(w.intersection(wire_4)) for w in wires_09 ]
    idx = matching.index(4)

    wire_9 = wires_09[idx]
    wire_9 = "".join(sorted(list(wire_9)))

    wire_to_digit[i][wire_9] = 9

    # Observation III: the last 6-wired digit is 0
    wire_0 = [wires for wires in wire_to_digit[i] if len(wires) == 6 and wire_to_digit[i][wires] == -1][0]
    wire_to_digit[i][wire_0] = 0

    # Observation IV: the remaining 5-wired digits are 2, 3, 5. From these, the wires of 3 and 5 are matching with the wires of 9, therefore 2 can be found by exclusion.
    wire_9 = set(wire_9) # the value is still in memory from Observation II.
    wires_235 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 5]
    
    matching = [ len(w.intersection(wire_9)) for w in wires_235 ]
    idx = matching.index(4)

    wire_2 = wires_235[idx]
    wire_2 = "".join(sorted(list(wire_2)))

    wire_to_digit[i][wire_2] = 2

    # Observation V: wires of 6 contain the wires of 5 entirely, but not 3
    wire_6 = set(wire_6)
    wires_35 = [set(wires) for wires in wire_to_digit[i] if len(wires) == 5 and wire_to_digit[i][wires] == -1]

    matching = [ len(w.intersection(wire_6)) for w in wires_35 ]
    idx = matching.index(5)

    wire_5 = wires_35[idx]
    wire_5 = "".join(sorted(list(wire_5)))

    wire_to_digit[i][wire_5] = 5

    # Observation VI: last value is 3
    wire_3 = [wires for wires in wire_to_digit[i] if len(wires) == 5 and wire_to_digit[i][wires] == -1][0]
    wire_to_digit[i][wire_3] = 3

# for mapping in wire_to_digit:
#     print(mapping)

### Now calculating outputs
outputs = []

for i, l in enumerate(lines):

    # same processing steps as on the left sections of the lines
    o = l[1].split(" ")
    o = [ "".join(sorted(w)) for w in o ]

    # apply decoding, format to one integer
    o = [ str(wire_to_digit[i][w]) for w in o ]
    o = int("".join(o))
    # print(o)

    outputs.append(o)

ans = sum(outputs)
print("Answer:", ans)

Answer: 456
Answer: 1091609
